# Задание

Представим, что меж


Для поиска оптимальниых мест 

Часть открытых данных есть, например, ан сайте archive.org:


# Процессинг данных

Загрузим данные.
Для удобной работы с этим документом преобразуем его в формат csv, кдалив строки, не содержащие координат - они не информативны для нас:

In [1]:
import csv

In [2]:
%%time
with open('checkins.dat') as input_file:
    newLines = []
    for line in input_file:
        newLine = [x.strip() for x in line.split('|')]
        if len(newLine) == 6 and newLine[3] and newLine[4]:
            newLines.append(newLine)

Wall time: 4.97 s


In [3]:
%%time
with open('checkins.csv', 'w') as output_file:
    file_writer = csv.writer(output_file)
    file_writer.writerows(newLines)

Wall time: 2.43 s


In [4]:
import pandas as pd
import numpy as np

In [5]:
%%time
data = pd.read_csv('checkins.csv', header = 0)

Wall time: 876 ms


In [6]:
len(data)

396634

In [7]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


Теперь необходимо кластерирозвать.....



Как мы помнив 20 баннеров нужно разместить

In [8]:
import sklearn.cluster as cluster

In [9]:
X = data.values[0:100000,3:5]

In [10]:
cluster1 = cluster.MeanShift(bandwidth=0.1)

In [12]:
%%time
cluster1.fit(X)

Wall time: 2min 36s


MeanShift(bandwidth=0.1)

In [16]:
labels = cluster1.labels_
cluster_centers = cluster1.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
n_clusters_

3231

In [17]:
cluster_centers[1]

array([  33.44943805, -112.00213969])

In [20]:
d = {}
for label in labels:
    if label not in d.keys():
        d[label] = 1
    else:
        d[label] +=1

In [22]:
count = 0
for key in d.keys():
    if d[key] > 15:
        count +=1
print(count)

592


In [26]:
clusters_select = np.ndarray(shape=(count,2))
clusters_select

array([[9.50186544e-312, 9.50287381e-312],
       [9.50031493e-312, 9.50265412e-312],
       [9.50265443e-312, 9.50265412e-312],
       ...,
       [9.50306533e-312, 9.50306533e-312],
       [9.50306482e-312, 9.50302582e-312],
       [9.50031493e-312, 9.50306553e-312]])

In [27]:
i=0
j=0
while i < len(cluster_centers):
    if d[i] > 15:
        clusters_select[j] = cluster_centers[i]
        j +=1
    i +=1

In [28]:
len(clusters_select)

592

In [29]:
clusters_select[0]

array([ 40.7177164 , -73.99183542])

Как ми помним, 20 баннеров надл разместить вбилзи офисов компании. Найдем на Google Maps по запросу "Carnival Cruise Line" адреса офисов:

33.751277, -118.188740 (Los Angeles)
25.867736, -80.324116 (Miami)
51.503016, -0.075479 (London)
52.378894, 4.885084 (Amsterdam)
39.366487, 117.036146 (Beijing)
-33.868457, 151.205134 (Sydney)

Осталось определить 20 ближайших к ним центров кластеров, тоесть посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.
Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала,  а в остальных точках значение достаточно велико.

Для сдачи задания нужно выбрать из получившихся 20 центров тот, который наимение удален от ближайшего к ним офиса. Ответ в этом задании - широта и долгота этого центра, записанные через пробел.

In [30]:
offices = np.ndarray(shape=(6,2))
offices[0] = np.array([33.751277, -118.188740])
offices[1] = np.array([25.867736, -80.324116])
offices[2] = np.array([51.503016, -0.075479])
offices[3] = np.array([52.378894, 4.885084])
offices[4] = np.array([39.366487, 117.036146])
offices[5] = np.array([-33.868457, 151.205134])

In [35]:
def distance(point1, point2):
    return((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)**0.5

In [37]:
distance(offices[0], clusters_select[123])

20.892897123756782

In [38]:
answer_index = 0
min_dist = 0
i = 0
while i < len(clusters_select):
    distances = [distance(xx, clusters_select[i]) for xx in offices]
    if min_dist == 0:
        min_dist = min(distances)
        answer_indes = i
    elif min_dist > min(distances):
        min_dist = min(distances)
        answer_index = i
    i +=1

In [39]:
answer_index

417

In [40]:
min_dist

0.007834758163107856

In [41]:
clusters_select[answer_index]

array([-33.86063043, 151.20477593])

In [44]:
def write_answer(center):
    with open('anser.txt','w') as f:
        f.write(str(center[0]) + ' ' + str(center[1]))

In [45]:
write_answer(clusters_select[answer_index])